# DETOXIFICATION

In [1]:
%pip install -q torch
%pip install -q transformers[torch]
%pip install -q googletrans==3.1.0a0
%pip install -q fastparquet
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

## Lectura de datos

Vamos a leer el dataset de entrenamiento. Este dataset esta formado por 400 frases tóxicas por idioma en 9 idiomas diferentes y sus versiones no tóxicas. Formaremos un dataset conjunto que contenga para cada frase, su idioma de origen, la frase tóxica y la frase de-toxificada.

In [3]:
splits = {'en': 'data/en-00000-of-00001.parquet', 'ru': 'data/ru-00000-of-00001.parquet', 'uk': 'data/uk-00000-of-00001.parquet', 'de': 'data/de-00000-of-00001.parquet', 'es': 'data/es-00000-of-00001.parquet', 'am': 'data/am-00000-of-00001.parquet', 'zh': 'data/zh-00000-of-00001.parquet', 'ar': 'data/ar-00000-of-00001.parquet', 'hi': 'data/hi-00000-of-00001.parquet'}
df_en = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["en"])
df_uk = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["uk"])
df_ru = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ru"])
df_de = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["de"])
df_es = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["es"])
df_am = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["am"])
df_zh = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["zh"])
df_ar = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["ar"])
df_hi = pd.read_parquet("hf://datasets/textdetox/multilingual_paradetox/" + splits["hi"])


In [4]:
df_total = pd.concat([df_en, df_uk, df_ru, df_de, df_es, df_am, df_zh, df_ar, df_hi], keys=['en','uk', 'ru', 'de', 'es', 'am', 'zh-cn', 'ar', 'hi']).reset_index(level=[0])

In [5]:
df_total.sample(3)

,level_0,toxic_sentence,neutral_sentence
367,es,Quien putas está RT esa mierda ?,Quien está RT esto?
199,en,cant you lazy bums do anything ?,Can't you do anything?
39,ar,اي والله هو غبي اخاف يشله السيل بقوله ينتبه 🥵,اي والله اخاف يشله السيل بقوله ينتبه 🥵


### Train-test split

In [6]:
train, test = train_test_split(df_total, train_size=0.8, random_state=33)

X_train = list(train['toxic_sentence'])
y_train = list(train['neutral_sentence'])

X_test = list(test['toxic_sentence'])
y_test = list(test['neutral_sentence'])
len(X_test)

720

## Pipeline de detoxificación

Nuestro modelo de detoxificación tendrá un pipeline que consistirá de los siguientes pasos:

1. **Traducción**: Traduciremos las frases del idioma original al inglés
2. **Detoxificación**: El modelo detoxificará la frase en inglés
3. **Retro-traducción**: La frase detoxificada será traducida del inglés al idioma original

### Traducción y retrotraducción

Usaremos la libreria de google translate para crear un modelo que traduzca un texto de un lenguaje a otro, pasados por argumentos.

In [7]:
from googletrans import Translator

translator = Translator()

In [8]:
def translate(text, source_lang, dest_lang):
    if source_lang == dest_lang:
        return text
    
    sol = translator.translate(text , src = source_lang, dest=dest_lang).text
    return sol

In [9]:
test_trad = df_total.sample(5).to_numpy()

for sentence in test_trad:
    print(f"Original sentence ({sentence[0]}): {sentence[1]}")
    translation = translate(sentence[1], sentence[0], "en")
    print(f"Transalation: {translation}")
    print("-"*50)

Original sentence (zh-cn): 同婚不能减少骗婚，基佬骗婚是因为想要繁衍后代，占两种便宜而已，不是来自社会压力，禁止同性恋繁衍后代就可以消灭骗婚。
Transalation: Homosexual marriage cannot reduce fraudulent marriages. Gays cheat in marriages because they want to reproduce and have two advantages. It does not come from social pressure. Forbidding homosexuals to reproduce will eliminate fraudulent marriages.
--------------------------------------------------
Original sentence (ru): так... один бот из пригожинской фашистской лахтопараши уже пошел в атаку... твой взвод идет следом? или ты в разведку вышел?
Transalation: so... one bot from Prigozhin's fascist Lakhtoparasha has already gone on the attack... is your platoon following? or did you go out on reconnaissance?
--------------------------------------------------
Original sentence (es): Los mejores y los más valientes.
Transalation: The best and the bravest.
--------------------------------------------------
Original sentence (de): warum lädt man eine derart strunzdumme Frau #Demirkan bei #Maisch

## Detoxificación

Ahora haremos un finetunning de un modelo de Huggingface para detoxificar el código.

In [10]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

num_training_layers = 2 # El número de capas que se entrenarán

# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas n capas del decoder
for param in model.decoder.block[-num_training_layers:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ESTADÍSTICAS DEL ENTRENAMIENTO:")
print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


ESTADÍSTICAS DEL ENTRENAMIENTO:
Total de parámetros: 76,961,152
Parámetros entrenables: 22,744,064
Parámetros congelados: 54,217,088


#### Creación de los dataset

In [11]:
class DetoxificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

In [12]:
X_train_tokenized = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
y_train_tokenized = tokenizer(y_train, truncation=True, padding=True, return_tensors="pt")

X_test_tokenized = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")
y_test_tokenized = tokenizer(y_test, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxificationDataset(X_train_tokenized, y_train_tokenized)
val_dataset = DetoxificationDataset(X_test_tokenized, y_test_tokenized)

training_args = TrainingArguments(
    output_dir='./checkpoints',
    eval_strategy="no", 
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-4,
    num_train_epochs=5,
    report_to=["none"],
    fp16=True, # acelerar entrenaminento 
)


## Entrenamiento

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# Entrenar el modelo
trainer.train()



C:\Users\mgonz\AppData\Local\Temp\ipykernel_13932\2669977037.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


KeyboardInterrupt: 